# Getting started

Ontology based atomic structure creation, manipulation, querying.

Imports

In [1]:
from atomrdf import KnowledgeGraph, System

The initial step is to create a Knowledge Graph

In [2]:
kg = KnowledgeGraph()

## Creation of structures

We will create three structures for the demonstration.

First a BCC Iron structure

In [3]:
struct_Fe = System.create.element.Fe(graph=kg)

NameError: name 'dm' is not defined

Note that we passed an argument `graph=kg` which ensures that when the structure is created, it is also added to the Graph automatically. We can visualise the graph.

In [ ]:
kg.visualise(hide_types=True)

Now a Si diamond structure

In [ ]:
struct_Si = System.create.element.Si(graph=kg)

Finally, an L12 $Ni_3 Al$ structure

In [ ]:
struct_l12 = System.create.lattice.l12(element=['Al', 'Ni'], 
                         lattice_constant=3.57, graph=kg)

In [ ]:
kg.visualise(hide_types=True, size=(60,30))

We can save the graph and reload it as needed

In [ ]:
kg.write('serial.ttl', format='ttl')

In [ ]:
kg = KnowledgeGraph(graph_file='serial.ttl')

In [ ]:
kg.n_samples

## Querying the graph

An example question would be, **what are the space group of all structures with 4 atoms?**


The corresponding SPARQL query looks like this:

In [ ]:
query = """
PREFIX cmso: <http://purls.helmholtz-metadaten.de/cmso/>
SELECT DISTINCT ?symbol
WHERE {
    ?sample cmso:hasNumberOfAtoms ?number .
    ?sample cmso:hasMaterial ?material .
    ?material cmso:hasStructure ?structure .
    ?structure cmso:hasSpaceGroup ?spgroup .
    ?spgroup cmso:hasSpaceGroupSymbol ?symbol .
FILTER (?number="4"^^xsd:integer)
}"""

In [ ]:
res = kg.query(query)

And print the results

In [ ]:
res

The query system can also be used without experience in SPARQL, or deep knowledge about the ontology terms. For example, **What are all the samples with Bravais lattice bcc?**

First how this looks like:

In [ ]:
res = kg.query_sample(kg.ontology.terms.cmso.hasAltName, 
             condition=(kg.ontology.terms.cmso.hasAltName=='bcc'))
res

As expected, there is only one sample, since Fe is the only bcc structure we added. We can extract the sample

In [ ]:
sample = res.AtomicScaleSample[0]

We can write this sample to a file, for example, a LAMMPS data format, to use it for further simulations

In [ ]:
kg.to_file(sample, 'bcc.data', format="lammps-data")

In [ ]:
! more bcc.data

We can also export as an ASE object

In [ ]:
aseobj = kg.to_file(sample, format="ase")

In [ ]:
aseobj